# Определение минимального объема выборки (числа экспериментов)

Пусть задана надежность $\gamma = 0.95$. Необходимо найти минимальное число экспериментов, позволяющих построить 95% доверительный интервал для величины $\overline{f_A}$, где $f_A$ - трудоемкость алгоритма А на входе фиксированной длины

In [2]:
import numpy as np
import random
import scipy.integrate as integrate
from scipy.stats import beta

def Damerau_Levenshtein_metric(str1, str2):
    f = 0 # счетчик функции трудоемкости
    N, M  = len(str1), len(str2)
    
    # инициализация матрицы D размером (N+1)x(M+1) сразу с базовым случаем
    D = [[i + j if i * j==0 else 0 for j in range(0, M+1)]
         for i in range(N + 1)]
    
    
    f += 2 # инициализация i и последнее сравнение
    for i in range(1, N+1):
        f += 2 # присваивание значения i и его сравнение (i < N+1?)
        f += 2 # инициализация j и последнее сравнение 
        for j in range(1, M+1):
            f += 2 # присваивание значения j и его сравнение (j < M+1?)
            f += 1 # сравнение элементов строк
            if str1[i-1] == str2[j-1]:
                f += 1 # присваивание
                D[i][j] = D[i-1][j-1] 
            else:
                f += 3 # сравнение в min и присваивание
                D[i][j] = 1 + min(D[i-1][j-1], D[i-1][j], D[i][j-1])
            
            
            
            f += 2 # сравнение i и j
            if (i > 1) and (j > 1):
                f += 2 # сравнение двух пар элементов строк
                if (str1[i-1] == str2[j-2]) and (str1[i-2] == str2[j-1]):
                    f += 2 # сравнение в min и присваивание
                    D[i][j] = min(D[i][j], D[i-2][j-2] + 1)
    
    
    return f, D[N][M]

# символы, из которых будут составляться слова
letters = 'ACGT'

def generation_random_strings(len1, len2, seed=None):
    if seed is not None:
        random.seed(seed)
        
    random_string_1 = ''.join(random.choice(letters) for i in range(len1))
    random_string_2 = ''.join(random.choice(letters) for i in range(len2))
    
    return random_string_1, random_string_2

In [3]:
N = 60
def min_number_of_experiments(k_start=50, sigma=0.95):
    k = k_start
    # предварительный этап
    F = []
    for i in range(k):
        str1, str2 = generation_random_strings(N, N)
        f, res = Damerau_Levenshtein_metric(str1, str2)
        F.append(f)
    F = np.array(sorted(F))
    
    # нормировка
    f_best_current = F[0]
    f_worst_current = F[k-1]
    divider = f_worst_current - f_best_current
    X = []
    for f_i in F:    
        x_i = (f_i - f_best_current) / divider
        X.append(x_i)
    X = np.array(X)
    # print(f'f_best = {f_best_current}, f_worst = {f_worst_current}')
    
    # вычисление выборочной средней и дисперсии
    M_emp = np.sum(X) / k
    D_emp = np.sum((X - M_emp)**2) / (k - 1)
    # print(f'M_emp = {M_emp}, D_emp={D_emp}')
    
    # определение параметров аппроксимирующего бета-распределения
    a = (M_emp / D_emp) * (M_emp - M_emp**2 - D_emp)
    b = ((1 - M_emp) / D_emp) * (M_emp - M_emp**2 - D_emp)
    # print(f'a={a}, b={b}')
    
    # длина интервала 2 * fi
    fi = round(0.025 * (f_worst_current - f_best_current))
    #print(f'fi={fi}')
    
    # вычисление пределов интегрирования
    t1 = (np.mean(F) - f_best_current - fi) / (f_worst_current - f_best_current)
    t2 = (np.mean(F) - f_best_current + fi) / (f_worst_current - f_best_current)

    # print(f't1={t1}, t2={t2}')
    
    # решение интегрального уравнения
    u = lambda n: a/(a+b) * (n*(a + b + 1) - 1)
    v = lambda n: b/(a+b) * (n*(a + b + 1) - 1)
    p = 0
    n = 10 # минимальный объем выборки
    while p < sigma:  
        n += 1
    
        # плотность бета-распределения -> подинтегральная функция
        beta_distrub = lambda x: beta.pdf(x, u(n), v(n))
        
        p, err = integrate.quad(beta_distrub, t1, t2)        

    return n

In [4]:
n1 = min_number_of_experiments(k_start=100)
n2 = min_number_of_experiments(n1)
while n2 >= n1:
    n1 = n2
    n2 = min_number_of_experiments(n1)
print(f'n2 = {n2}, n1 = {n1}')

n2 = 169, n1 = 255


## Таким образом, нужное количество экспериментов - 169